In [ ]:
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt

df['Date'] = pd.to_datetime(df['Date'])


traffic_data_hourly = df.groupby(pd.Grouper(key='Date', freq='H')).agg({
    'Vehicle_count': 'mean', 
    'Average_time_to_cross_junction': 'mean', 
    'Number_of_signals': 'mean'
}).reset_index()

traffic_data_hourly.rename(columns={'Date': 'ds', 'Vehicle_count': 'y'}, inplace=True)

model = Prophet()


model.fit(traffic_data_hourly)


def plot_between_dates(start_date, end_date):
    future = model.make_future_dataframe(periods=30*24, freq='H')  
    

    forecast = model.predict(future)
    

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    

    mask = (forecast['ds'] >= start_date) & (forecast['ds'] <= end_date)
    filtered_forecast = forecast[mask]

    if filtered_forecast.empty:
        print(f"No data available between {start_date} and {end_date}")
        return


    plt.figure(figsize=(10, 6))
    plt.plot(filtered_forecast['ds'], filtered_forecast['yhat'], label='Predicted Vehicle Count', color='blue')
    plt.fill_between(filtered_forecast['ds'], filtered_forecast['yhat_lower'], filtered_forecast['yhat_upper'], color='lightblue', label='Confidence Interval')
    plt.title(f"Vehicle Count Forecast from {start_date.date()} to {end_date.date()}")
    plt.xlabel("Date")
    plt.ylabel("Vehicle Count")
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:
start_date = '2023-8-01'  
end_date = '2023-8-30'
plot_between_dates(start_date, end_date)
